In [65]:
import pandas as pa
import numpy as np

# For preprocessing the data
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
# To split the dataset into train and test datasets
from sklearn.model_selection import train_test_split
# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import GaussianNB
# To model the Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
# To model the Decision Tree classifier
from sklearn.tree import DecisionTreeClassifier
# To model the Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
# To model the Linear SVC classifier
from sklearn.svm import LinearSVC
# To model the K Nearest Neighbors classifier
from sklearn.neighbors import KNeighborsClassifier
# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score
# To generate classification report
from sklearn.metrics import classification_report

from sklearn.cross_validation import cross_val_score

In [66]:
user_df = pa.read_csv('test_users.csv', delimiter=' *, *', engine='python')
user_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,NaN,FEMALE,35.0,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [67]:
user_df = pa.read_csv('train_users_2.csv', delimiter=' *, *', engine='python')
user_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [68]:
age_gender_df = pa.read_csv('age_gender_bkts.csv', delimiter=' *, *', engine='python')
age_gender_df.head()


,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0
3,85-89,AU,male,118.0,2015.0
4,80-84,AU,male,199.0,2015.0


In [69]:
age_gender_df.drop('year', axis=1, inplace=True)

In [70]:
def convert_df(country):
    country_df = age_gender_df[age_gender_df.country_destination==country]
    country_df = country_df.groupby(country_df.age_bucket).sum()
    pop_sum = country_df.population_in_thousands.sum()
    country_df.population_in_thousands/=pop_sum
    country_df.rename(columns={'population_in_thousands':country+'_pop'}, inplace=True)
    return country_df

country_list = age_gender_df.country_destination.unique()
country_df_dict = {}
for country in country_list:
    country_df_dict[country] = convert_df(country)
au_df = country_df_dict['AU']
au_df

,AU_pop
age_bucket,
0-4,0.067090
10-14,0.060611
100+,0.000209
15-19,0.062910
20-24,0.067174
25-29,0.072984
30-34,0.072984
35-39,0.066798
40-44,0.069306


In [71]:
age_buckets = age_gender_df.age_bucket.unique()
age_buckets

array(['100+', '95-99', '90-94', '85-89', '80-84', '75-79', '70-74',
       '65-69', '60-64', '55-59', '50-54', '45-49', '40-44', '35-39',
       '30-34', '25-29', '20-24', '15-19', '10-14', '5-9', '0-4'], dtype=object)

In [72]:


def get_age_range(age_value):

    for age_range in age_buckets:
        if age_range.find('+')!=-1:
            if age_value>=100:
                return age_range
        else:
            a,b = age_range.split('-')
            if age_value>=int(a) and age_value<=int(b):
                return age_range

    return None

In [73]:
user_df["age_bucket"] = user_df["age"].apply(get_age_range)


In [74]:
user_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,age_bucket
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,None
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,35-39
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,55-59
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,40-44
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,40-44


In [75]:
user_df.loc[user_df.age > 90, 'age'] = np.NaN
user_df.loc[user_df.age < 15, 'age'] = np.NaN
user_df['age'].fillna(-1, inplace=True)
user_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,age_bucket
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,-1.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,None
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,35-39
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,55-59
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,40-44
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,40-44


In [76]:
lowercase = lambda x: x.lower()

user_df['gender'] = user_df['gender'].apply(lowercase)

In [77]:
#user_df2 = pa.merge(user_df,age_gender_df, on=['age_bucket','country_destination','gender'])
#del user_df2['year']
user_df2 = user_df
for country, country_df in country_df_dict.items():
    user_df2 = user_df2.join(country_df, on='age_bucket')
#user_df2 = pa.merge(user_df, country_df_dict['AU'], on=['age_bucket'])
user_df2.head()
#country_df_dict['AU'].head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,age_bucket,AU_pop,CA_pop,DE_pop,ES_pop,FR_pop,GB_pop,IT_pop,NL_pop,PT_pop,US_pop
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,-1.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,820tgsjxq7,2011-05-25,20090523174809,NaN,male,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,35-39,0.066798,0.066377,0.058744,0.086499,0.056907,0.061764,0.068804,0.058167,0.078518,0.063608
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,female,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,55-59,0.060737,0.072259,0.072418,0.063767,0.062539,0.061341,0.066154,0.068910,0.066830,0.067520
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,female,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,40-44,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,40-44,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950


In [78]:
user_df2.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,age_bucket,AU_pop,CA_pop,DE_pop,ES_pop,FR_pop,GB_pop,IT_pop,NL_pop,PT_pop,US_pop
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,-1.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,820tgsjxq7,2011-05-25,20090523174809,NaN,male,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,35-39,0.066798,0.066377,0.058744,0.086499,0.056907,0.061764,0.068804,0.058167,0.078518,0.063608
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,female,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,55-59,0.060737,0.072259,0.072418,0.063767,0.062539,0.061341,0.066154,0.068910,0.066830,0.067520
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,female,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,40-44,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,40-44,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950


In [79]:
countries_df = pa.read_csv('countries.csv', delimiter=' *, *', engine='python')
countries_df.head()

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06


In [80]:
#user_df3 = pa.merge(user_df2,countries_df, on=['country_destination'])

In [81]:
#user_df3.head()
user_df3 = user_df2

In [82]:
user_df3.columns

Index(['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'country_destination', 'age_bucket', 'AU_pop', 'CA_pop', 'DE_pop', 'ES_pop', 'FR_pop', 'GB_pop', 'IT_pop', 'NL_pop', 'PT_pop', 'US_pop'], dtype='object')

In [83]:
user_df3["date_account_created"] = pa.to_datetime(user_df3["date_account_created"])
user_df3["year"] = user_df3["date_account_created"].dt.year
user_df3["month"] = user_df3["date_account_created"].dt.month

In [84]:
user_df3.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,age_bucket,AU_pop,CA_pop,DE_pop,ES_pop,FR_pop,GB_pop,IT_pop,NL_pop,PT_pop,US_pop,year,month
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,-1.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010,6
1,820tgsjxq7,2011-05-25,20090523174809,NaN,male,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,35-39,0.066798,0.066377,0.058744,0.086499,0.056907,0.061764,0.068804,0.058167,0.078518,0.063608,2011,5
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,female,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,55-59,0.060737,0.072259,0.072418,0.063767,0.062539,0.061341,0.066154,0.068910,0.066830,0.067520,2010,9
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,female,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,40-44,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950,2011,12
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,40-44,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950,2010,9


In [85]:
def calc_fast_features(df):
    df["date_account_created"] = pa.to_datetime(df["date_account_created"], format='%Y-%m-%d', errors="coerce")
    df["date_first_booking"] = pa.to_datetime(df["date_first_booking"], format='%Y-%m-%d', errors="coerce")
    
    props = {}
    for prop in ["month", "day", "dayofweek", "quarter"]:
        props[prop] = getattr(df["date_first_booking"].dt, prop)
    
    carryover =[p for p in df.columns ]#if p not in ["date_first_booking"]]
    for prop in carryover:
        props[prop] = df[prop]
    
    date_props = ["month", "day", "dayofweek", "quarter"]
    for prop in date_props:
        props["dfb_{0}".format(prop)] = getattr(df["date_first_booking"].dt, prop)
    props["account_activity_span"] = (df["date_first_booking"] - df["date_account_created"] ).astype('timedelta64[D]')
        
    ret = pa.DataFrame(props)
    return ret

user_df4 = calc_fast_features(user_df3)
user_df4.fillna(-1, inplace=True)

In [86]:
pa.set_option('display.height', 1000)
pa.set_option('display.max_rows', 500)
pa.set_option('display.max_columns', 500)
pa.set_option('display.width', 1000)

user_df4


,AU_pop,CA_pop,DE_pop,ES_pop,FR_pop,GB_pop,IT_pop,NL_pop,PT_pop,US_pop,account_activity_span,affiliate_channel,affiliate_provider,age,age_bucket,country_destination,date_account_created,date_first_booking,day,dayofweek,dfb_day,dfb_dayofweek,dfb_month,dfb_quarter,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,month,quarter,signup_app,signup_flow,signup_method,timestamp_first_active,year
0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,direct,direct,-1.0,-1,NDF,2010-06-28,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,6,-1.0,Web,0,facebook,20090319043255,2010
1,0.066798,0.066377,0.058744,0.086499,0.056907,0.061764,0.068804,0.058167,0.078518,0.063608,-1.0,seo,google,38.0,35-39,NDF,2011-05-25,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,untracked,Chrome,Mac Desktop,male,820tgsjxq7,en,5,-1.0,Web,0,facebook,20090523174809,2011
2,0.060737,0.072259,0.072418,0.063767,0.062539,0.061341,0.066154,0.068910,0.066830,0.067520,-57.0,direct,direct,56.0,55-59,US,2010-09-28,2010-08-02 00:00:00,2.0,0.0,2.0,0.0,8.0,3.0,untracked,IE,Windows Desktop,female,4ft3gnwmtx,en,9,3.0,Web,3,basic,20090609231247,2010
3,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950,278.0,direct,direct,42.0,40-44,other,2011-12-05,2012-09-08 00:00:00,8.0,5.0,8.0,5.0,9.0,3.0,untracked,Firefox,Mac Desktop,female,bjjt8pjhuk,en,12,3.0,Web,0,facebook,20091031060129,2011
4,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950,-208.0,direct,direct,41.0,40-44,US,2010-09-14,2010-02-18 00:00:00,18.0,3.0,18.0,3.0,2.0,1.0,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,9,1.0,Web,0,basic,20091208061105,2010
5,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.0,other,other,-1.0,-1,US,2010-01-01,2010-01-02 00:00:00,2.0,5.0,2.0,5.0,1.0,1.0,omg,Chrome,Mac Desktop,-unknown-,osr2jwljor,en,1,1.0,Web,0,basic,20100101215619,2010
6,0.065669,0.068272,0.083356,0.079677,0.067925,0.072838,0.081053,0.076389,0.073617,0.064937,3.0,other,craigslist,46.0,45-49,US,2010-01-02,2010-01-05 00:00:00,5.0,1.0,5.0,1.0,1.0,1.0,untracked,Safari,Mac Desktop,female,lsw9q7uk0j,en,1,1.0,Web,0,basic,20100102012558,2010
7,0.065669,0.068272,0.083356,0.079677,0.067925,0.072838,0.081053,0.076389,0.073617,0.064937,10.0,direct,direct,47.0,45-49,US,2010-01-03,2010-01-13 00:00:00,13.0,2.0,13.0,2.0,1.0,1.0,omg,Safari,Mac Desktop,female,0d01nltbrs,en,1,1.0,Web,0,basic,20100103191905,2010
8,0.065878,0.077807,0.086323,0.073300,0.067048,0.070896,0.076785,0.076092,0.073051,0.068975,206.0,other,craigslist,50.0,50-54,US,2010-01-04,2010-07-29 00:00:00,29.0,3.0,29.0,3.0,7.0,3.0,untracked,Safari,Mac Desktop,female,a1vcnhxeij,en,1,3.0,Web,0,basic,20100104004211,2010
9,0.065669,0.068272,0.083356,0.079677,0.067925,0.072838,0.081053,0.076389,0.073617,0.064937,0.0,other,craigslist,46.0,45-49,US,2010-01-04,2010-01-04 00:00:00,4.0,0.0,4.0,0.0,1.0,1.0,omg,Firefox,Mac Desktop,-unknown-,6uh8zyj2gn,en,1,1.0,Web,0,basic,20100104023758,2010


In [87]:
user_df4.columns

Index(['AU_pop', 'CA_pop', 'DE_pop', 'ES_pop', 'FR_pop', 'GB_pop', 'IT_pop', 'NL_pop', 'PT_pop', 'US_pop', 'account_activity_span', 'affiliate_channel', 'affiliate_provider', 'age', 'age_bucket', 'country_destination', 'date_account_created', 'date_first_booking', 'day', 'dayofweek', 'dfb_day', 'dfb_dayofweek', 'dfb_month', 'dfb_quarter', 'first_affiliate_tracked', 'first_browser', 'first_device_type', 'gender', 'id', 'language', 'month', 'quarter', 'signup_app', 'signup_flow', 'signup_method', 'timestamp_first_active', 'year'], dtype='object')

In [88]:
user_df4.describe(include= 'all')

,AU_pop,CA_pop,DE_pop,ES_pop,FR_pop,GB_pop,IT_pop,NL_pop,PT_pop,US_pop,account_activity_span,affiliate_channel,affiliate_provider,age,age_bucket,country_destination,date_account_created,date_first_booking,day,dayofweek,dfb_day,dfb_dayofweek,dfb_month,dfb_quarter,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,month,quarter,signup_app,signup_flow,signup_method,timestamp_first_active,year
count,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451,213451,213451.000000,213451.0,213451,213451,213451.0,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451.000000,213451,213451,213451,213451,213451,213451,213451.000000,213451.000000,213451,213451.000000,213451,2.134510e+05,213451.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,18,NaN,21.0,12,1634,1977.0,NaN,NaN,NaN,NaN,NaN,NaN,8,52,9,4,213451,25,NaN,NaN,4,NaN,3,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,direct,direct,NaN,-1.0,NDF,2014-05-13 00:00:00,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,untracked,Chrome,Mac Desktop,-unknown-,9hdh8rm1cb,en,NaN,NaN,Web,NaN,basic,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,137727,137426,NaN,87990.0,124543,674,124543.0,NaN,NaN,NaN,NaN,NaN,NaN,109232,63845,89600,95688,1,206314,NaN,NaN,182717,NaN,152897,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-06-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,-0.372834,-0.373213,-0.375555,-0.372669,-0.376234,-0.374501,-0.375961,-0.376111,-0.373624,-0.374349,17.894683,NaN,NaN,20.563923,NaN,NaN,NaN,NaN,5.932879,0.584139,5.932879,0.584139,1.961752,0.400893,NaN,NaN,NaN,NaN,NaN,NaN,6.022459,0.400893,NaN,3.267387,NaN,2.013085e+13,2013.023846
std,0.525301,0.524976,0.523035,0.525517,0.522433,0.523892,0.522709,0.522549,0.524664,0.524020,60.737809,NaN,NaN,20.454730,NaN,NaN,NaN,NaN,9.952020,2.245902,9.952020,2.245902,4.062306,1.789813,NaN,NaN,NaN,NaN,NaN,NaN,3.236690,1.789813,NaN,7.637707,NaN,9.253717e+09,0.938489
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-349.000000,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,-1.000000,NaN,0.000000,NaN,2.009032e+13,2010.000000
25%,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,-1.000000,NaN,0.000000,NaN,2.012123e+13,2012.000000


In [89]:
user_df4.columns

Index(['AU_pop', 'CA_pop', 'DE_pop', 'ES_pop', 'FR_pop', 'GB_pop', 'IT_pop', 'NL_pop', 'PT_pop', 'US_pop', 'account_activity_span', 'affiliate_channel', 'affiliate_provider', 'age', 'age_bucket', 'country_destination', 'date_account_created', 'date_first_booking', 'day', 'dayofweek', 'dfb_day', 'dfb_dayofweek', 'dfb_month', 'dfb_quarter', 'first_affiliate_tracked', 'first_browser', 'first_device_type', 'gender', 'id', 'language', 'month', 'quarter', 'signup_app', 'signup_flow', 'signup_method', 'timestamp_first_active', 'year'], dtype='object')

In [90]:
user_df4.isnull().sum()

AU_pop                     0
CA_pop                     0
DE_pop                     0
ES_pop                     0
FR_pop                     0
GB_pop                     0
IT_pop                     0
NL_pop                     0
PT_pop                     0
US_pop                     0
account_activity_span      0
affiliate_channel          0
affiliate_provider         0
age                        0
age_bucket                 0
country_destination        0
date_account_created       0
date_first_booking         0
day                        0
dayofweek                  0
dfb_day                    0
dfb_dayofweek              0
dfb_month                  0
dfb_quarter                0
first_affiliate_tracked    0
first_browser              0
first_device_type          0
gender                     0
id                         0
language                   0
month                      0
quarter                    0
signup_app                 0
signup_flow                0
signup_method 

In [91]:
del user_df4['date_first_booking']
del user_df4['date_account_created']

In [92]:
user_df4.head()

,AU_pop,CA_pop,DE_pop,ES_pop,FR_pop,GB_pop,IT_pop,NL_pop,PT_pop,US_pop,account_activity_span,affiliate_channel,affiliate_provider,age,age_bucket,country_destination,day,dayofweek,dfb_day,dfb_dayofweek,dfb_month,dfb_quarter,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,month,quarter,signup_app,signup_flow,signup_method,timestamp_first_active,year
0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,direct,direct,-1.0,-1,NDF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,6,-1.0,Web,0,facebook,20090319043255,2010
1,0.066798,0.066377,0.058744,0.086499,0.056907,0.061764,0.068804,0.058167,0.078518,0.063608,-1.0,seo,google,38.0,35-39,NDF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,untracked,Chrome,Mac Desktop,male,820tgsjxq7,en,5,-1.0,Web,0,facebook,20090523174809,2011
2,0.060737,0.072259,0.072418,0.063767,0.062539,0.061341,0.066154,0.068910,0.066830,0.067520,-57.0,direct,direct,56.0,55-59,US,2.0,0.0,2.0,0.0,8.0,3.0,untracked,IE,Windows Desktop,female,4ft3gnwmtx,en,9,3.0,Web,3,basic,20090609231247,2010
3,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950,278.0,direct,direct,42.0,40-44,other,8.0,5.0,8.0,5.0,9.0,3.0,untracked,Firefox,Mac Desktop,female,bjjt8pjhuk,en,12,3.0,Web,0,facebook,20091031060129,2011
4,0.069306,0.065457,0.062886,0.084232,0.068372,0.066541,0.079140,0.069148,0.078612,0.062950,-208.0,direct,direct,41.0,40-44,US,18.0,3.0,18.0,3.0,2.0,1.0,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,9,1.0,Web,0,basic,20091208061105,2010


In [93]:
del user_df4['id']

In [94]:
#user_df_final = pa.get_dummies(user_df4, columns=["affiliate_channel","affiliate_provider","age_bucket",
#                                                  "destination_language","first_affiliate_tracked",
#                                                 "first_browser","first_device_type","gender","signup_app","signup_method",
#                                                  "language"]
#                            , prefix=["affiliate_channel","affiliate_provider","age_bucket",
#                                     "destination_language","first_affiliate_tracked","first_browser",
#                                     "first_device_type","gender","signup_app","signup_method","language"])

user_df_final = pa.get_dummies(user_df4, columns=["affiliate_channel","affiliate_provider","age_bucket",
                                                  "first_affiliate_tracked",
                                                 "first_browser","first_device_type","gender","signup_app","signup_method",
                                                  "language"]
                            , prefix=["affiliate_channel","affiliate_provider","age_bucket",
                                     "first_affiliate_tracked","first_browser",
                                     "first_device_type","gender","signup_app","signup_method","language"])

user_df_final.head()

,AU_pop,CA_pop,DE_pop,ES_pop,FR_pop,GB_pop,IT_pop,NL_pop,PT_pop,US_pop,account_activity_span,age,country_destination,day,dayofweek,dfb_day,dfb_dayofweek,dfb_month,dfb_quarter,month,quarter,signup_flow,timestamp_first_active,year,affiliate_channel_api,affiliate_channel_content,affiliate_channel_direct,affiliate_channel_other,affiliate_channel_remarketing,affiliate_channel_sem-brand,affiliate_channel_sem-non-brand,affiliate_channel_seo,affiliate_provider_baidu,affiliate_provider_bing,affiliate_provider_craigslist,affiliate_provider_daum,affiliate_provider_direct,affiliate_provider_email-marketing,affiliate_provider_facebook,affiliate_provider_facebook-open-graph,affiliate_provider_google,affiliate_provider_gsp,affiliate_provider_meetup,affiliate_provider_naver,affiliate_provider_other,affiliate_provider_padmapper,affiliate_provider_vast,affiliate_provider_wayn,affiliate_provider_yahoo,affiliate_provider_yandex,age_bucket_-1,age_bucket_0-4,age_bucket_100+,age_bucket_15-19,age_bucket_20-24,age_bucket_25-29,age_bucket_30-34,age_bucket_35-39,age_bucket_40-44,age_bucket_45-49,age_bucket_5-9,age_bucket_50-54,age_bucket_55-59,age_bucket_60-64,age_bucket_65-69,age_bucket_70-74,age_bucket_75-79,age_bucket_80-84,age_bucket_85-89,age_bucket_90-94,age_bucket_95-99,first_affiliate_tracked_-1,first_affiliate_tracked_linked,first_affiliate_tracked_local ops,first_affiliate_tracked_marketing,first_affiliate_tracked_omg,first_affiliate_tracked_product,first_affiliate_tracked_tracked-other,first_affiliate_tracked_untracked,first_browser_-unknown-,first_browser_AOL Explorer,first_browser_Android Browser,first_browser_Apple Mail,first_browser_Arora,first_browser_Avant Browser,first_browser_BlackBerry Browser,first_browser_Camino,first_browser_Chrome,first_browser_Chrome Mobile,first_browser_Chromium,first_browser_CometBird,first_browser_Comodo Dragon,first_browser_Conkeror,first_browser_CoolNovo,first_browser_Crazy Browser,first_browser_Epic,first_browser_Firefox,first_browser_Flock,first_browser_Google Earth,first_browser_Googlebot,first_browser_IE,first_browser_IE Mobile,first_browser_IceDragon,first_browser_IceWeasel,first_browser_Iron,first_browser_Kindle Browser,first_browser_Maxthon,first_browser_Mobile Firefox,first_browser_Mobile Safari,first_browser_Mozilla,first_browser_NetNewsWire,first_browser_OmniWeb,first_browser_Opera,first_browser_Opera Mini,first_browser_Opera Mobile,first_browser_Outlook 2007,first_browser_PS Vita browser,first_browser_Pale Moon,first_browser_Palm Pre web browser,first_browser_RockMelt,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser,first_device_type_Android Phone,first_device_type_Android Tablet,first_device_type_Desktop (Other),first_device_type_Mac Desktop,first_device_type_Other/Unknown,first_device_type_SmartPhone (Other),first_device_type_Windows Desktop,first_device_type_iPad,first_device_type_iPhone,gender_-unknown-,gender_female,gender_male,gender_other,signup_app_Android,signup_app_Moweb,signup_app_Web,signup_app_iOS,signup_method_basic,signup_method_facebook,signup_method_google,language_ca,language_cs,language_da,language_de,language_el,language_en,language_es,language_fi,language_fr,language_hr,language_hu,language_id,language_is,language_it,language_ja,language_ko,language_nl,language_no,language_pl,language_pt,language_ru,language_sv,language_th,language_tr,language_zh
0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,NDF,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,6,-1.0,0,20090319043255,2010,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,

In [95]:
user_df_final.columns

Index(['AU_pop', 'CA_pop', 'DE_pop', 'ES_pop', 'FR_pop', 'GB_pop', 'IT_pop', 'NL_pop', 'PT_pop', 'US_pop',
       ...
       'language_ko', 'language_nl', 'language_no', 'language_pl', 'language_pt', 'language_ru', 'language_sv', 'language_th', 'language_tr', 'language_zh'], dtype='object', length=176)

In [96]:
feature_cols = user_df_final.columns.tolist()
# remove income from feature list
feature_cols.remove('country_destination')
X = user_df_final[feature_cols]  
y = user_df_final['country_destination']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [97]:

#del X_test["lat_destination"]
#del X_test["lng_destination"]
#del X_test["distance_km"]
#del X_test["destination_km2"]
#del X_test["destination_language"]
#del X_test["language_levenshtein_distance"]


In [98]:
my_gnb = GaussianNB()
my_gnb.fit(X_train, y_train)
y_predict_gnb = my_gnb.predict(X_test)

In [99]:
print('accuracy of Gaussian Naive Bayes is', accuracy_score(y_test, y_predict_gnb))

accuracy of Gaussian Naive Bayes is 0.579955025298


In [100]:
my_rf = RandomForestClassifier(n_estimators = 10, bootstrap = True, random_state=2)
my_rf.fit(X_train, y_train)
y_predict_rf = my_rf.predict(X_test)

In [101]:
print('accuracy of Random Forest is', accuracy_score(y_test, y_predict_rf))

accuracy of Random Forest is 0.864872884003


In [102]:
# Using Decision Tree classifier
my_dt = DecisionTreeClassifier()
my_dt.fit(X_train, y_train)
y_predict_dt = my_dt.predict(X_test)
print('accuracy of Decision Tree is', accuracy_score(y_test, y_predict_dt))



accuracy of Decision Tree is 0.788400274845


In [103]:
my_decision_tree = DecisionTreeClassifier()
scores = cross_val_score(my_decision_tree, X_train, y_train, cv=10)
accuracy_cv = scores.mean()
print(accuracy_cv)

0.788200748183


In [ ]:
# Using K Nearest Neighbors classifier (k = 3)
my_knn = KNeighborsClassifier(n_neighbors=7)
my_knn.fit(X_train, y_train)
y_predict_knn = my_knn.predict(X_test)
print('accuracy of K Nearest Neighbors is', accuracy_score(y_test, y_predict_knn))

accuracy of K Nearest Neighbors is 0.540539696421


In [ ]:
scores = cross_val_score(my_rf, X, y, cv=10)
accuracy_cv = scores.mean()
print(accuracy_cv)